In [12]:
import pandas as pd
import ast

In [13]:
df1 = pd.read_csv('..//data/processed/cve_fc_vcc_mapping.csv')
df2 = pd.read_csv('..//data/processed/function_info.csv')
df2

,hash,name,filename,num_lines_of_code,complexity,token_count,parameters,signature,start_line,end_line,length,top_nesting_level,code,before_change
0,6c4841c2b6c32a134f9f36e5e08857138cc12b10,altivec_unavailable_exception,traps.c,10,3,57,['regs'],altivec_unavailable_exception( struct pt_regs ...,901,914,14,0,void altivec_unavailable_exception(struct pt_r...,True
1,6c4841c2b6c32a134f9f36e5e08857138cc12b10,altivec_unavailable_exception,traps.c,10,2,57,['regs'],altivec_unavailable_exception( struct pt_regs ...,901,913,13,0,void altivec_unavailable_exception(struct pt_r...,False
2,f0ec1aaf54caddd21c259aea8b2ecfbde4ee4fb9,xacct_add_tsk,tsacct.c,13,2,112,"['stats', 'p']","xacct_add_tsk( struct taskstats * stats , stru...",81,95,15,0,"void xacct_add_tsk(struct taskstats *stats, st...",True
3,f0ec1aaf54caddd21c259aea8b2ecfbde4ee4fb9,xacct_add_tsk,tsacct.c,16,2,123,"['stats', 'p']","xacct_add_tsk( struct taskstats * stats , stru...",81,99,19,0,"void xacct_add_tsk(struct taskstats *stats, st...",False
4,2a2f11c227bdf292b3a2900ad04139d301b56ac4,netlbl_cipsov4_add_common,netlabel_cipso_v4.c,21,5,127,"['info', 'doi_def']",netlbl_cipsov4_add_common( struct genl_info * ...,117,141,25,0,static int netlbl_cipsov4_add_common(struct ge...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15391,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInitTokenized,Formatter.c,25,1,194,"['buffer', 'first_token', 'user_buffer', 'leng...",ZydisFormatterBufferInitTokenized( ZydisFormat...,80,108,29,0,void ZydisFormatterBufferInitTokenized(ZydisFo...,False
15392,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInitTokenized,Formatter.c,22,1,164,"['buffer', 'first_token', 'user_buffer', 'leng...",ZydisFormatterBufferInitTokenized( ZydisFormat...,75,99,25,0,void ZydisFormatterBufferInitTokenized(ZydisFo...,True
15393,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInit,Formatter.c,19,1,138,"['buffer', 'user_buffer', 'length']",ZydisFormatterBufferInit( ZydisFormatterBuffer...,58,78,21,0,void ZydisFormatterBufferInit(ZydisFormatterBu...,False
15394,fa6a76050d82894365dfe32916903ef7fee3ffcd,guestfs___first_line_of_file,inspect-fs.c,23,5,127,"['g', 'filename']","guestfs___first_line_of_file( guestfs_h * g , ...",545,577,33,0,"guestfs___first_line_of_file (guestfs_h *g, co...",True


In [14]:
# Function to convert string-formatted list to actual list
def convert_to_list(vcc_hash_str):
    if pd.isna(vcc_hash_str):
        return []
    try:
        # Convert the string to an actual list
        return ast.literal_eval(vcc_hash_str.strip())
    except (ValueError, SyntaxError):
        return []

# Apply the function to the 'vcc_hash' column to convert it to a list
df1['vcc_hash'] = df1['vcc_hash'].apply(convert_to_list)

# Create new columns in df2
df2['commit_type'] = ''
df2['cwe_id'] = ''
df2['repo_url'] = ''

# Iterate over each row in df1
for index, row in df1.iterrows():
    # Handle FC hashes
    fc_matches = df2['hash'] == row['fc_hash']
    df2.loc[fc_matches, 'commit_type'] = 'FC'
    df2.loc[fc_matches, 'cwe_id'] = row['cwe_id']
    df2.loc[fc_matches, 'repo_url'] = row['repo_url']

    # Handle VCC hashes
    vcc_hashes = row['vcc_hash']
    for vcc_hash in vcc_hashes:
        if pd.notna(vcc_hash):
            vcc_matches = df2['hash'] == vcc_hash
            df2.loc[vcc_matches, 'commit_type'] = 'VCC'
            df2.loc[vcc_matches, 'cwe_id'] = row['cwe_id']
            df2.loc[vcc_matches, 'repo_url'] = row['repo_url']

# Display the updated df2
df2

,hash,name,filename,num_lines_of_code,complexity,token_count,parameters,signature,start_line,end_line,length,top_nesting_level,code,before_change,commit_type,cwe_id,repo_url
0,6c4841c2b6c32a134f9f36e5e08857138cc12b10,altivec_unavailable_exception,traps.c,10,3,57,['regs'],altivec_unavailable_exception( struct pt_regs ...,901,914,14,0,void altivec_unavailable_exception(struct pt_r...,True,FC,CWE-19,https://github.com/torvalds/linux
1,6c4841c2b6c32a134f9f36e5e08857138cc12b10,altivec_unavailable_exception,traps.c,10,2,57,['regs'],altivec_unavailable_exception( struct pt_regs ...,901,913,13,0,void altivec_unavailable_exception(struct pt_r...,False,FC,CWE-19,https://github.com/torvalds/linux
2,f0ec1aaf54caddd21c259aea8b2ecfbde4ee4fb9,xacct_add_tsk,tsacct.c,13,2,112,"['stats', 'p']","xacct_add_tsk( struct taskstats * stats , stru...",81,95,15,0,"void xacct_add_tsk(struct taskstats *stats, st...",True,FC,CWE-399,https://github.com/torvalds/linux
3,f0ec1aaf54caddd21c259aea8b2ecfbde4ee4fb9,xacct_add_tsk,tsacct.c,16,2,123,"['stats', 'p']","xacct_add_tsk( struct taskstats * stats , stru...",81,99,19,0,"void xacct_add_tsk(struct taskstats *stats, st...",False,FC,CWE-399,https://github.com/torvalds/linux
4,2a2f11c227bdf292b3a2900ad04139d301b56ac4,netlbl_cipsov4_add_common,netlabel_cipso_v4.c,21,5,127,"['info', 'doi_def']",netlbl_cipsov4_add_common( struct genl_info * ...,117,141,25,0,static int netlbl_cipsov4_add_common(struct ge...,True,FC,CWE-119,https://github.com/torvalds/linux
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15391,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInitTokenized,Formatter.c,25,1,194,"['buffer', 'first_token', 'user_buffer', 'leng...",ZydisFormatterBufferInitTokenized( ZydisFormat...,80,108,29,0,void ZydisFormatterBufferInitTokenized(ZydisFo...,False,FC,CWE-908,https://github.com/zyantific/zydis
15392,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInitTokenized,Formatter.c,22,1,164,"['buffer', 'first_token', 'user_buffer', 'leng...",ZydisFormatterBufferInitTokenized( ZydisFormat...,75,99,25,0,void ZydisFormatterBufferInitTokenized(ZydisFo...,True,FC,CWE-908,https://github.com/zyantific/zydis
15393,55dd08c210722aed81b38132f5fd4a04ec1943b5,ZydisFormatterBufferInit,Formatter.c,19,1,138,"['buffer', 'user_buffer', 'length']",ZydisFormatterBufferInit( ZydisFormatterBuffer...,58,78,21,0,void ZydisFormatterBufferInit(ZydisFormatterBu...,False,FC,CWE-908,https://github.com/zyantific/zydis
15394,fa6a76050d82894365dfe32916903ef7fee3ffcd,guestfs___first_line_of_file,inspect-fs.c,23,5,127,"['g', 'filename']","guestfs___first_line_of_file( guestfs_h * g , ...",545,577,33,0,"guestfs___first_line_of_file (guestfs_h *g, co...",True,FC,NVD-CWE-Other,https://github.com/libguestfs/libguestfs


In [15]:
vul_count = df2['before_change'].value_counts()
vul_count

before_change
False    9120
True     6276
Name: count, dtype: int64

In [16]:
vul_rows_df = df2[df2['before_change'] == True]
cwe_id_count = vul_rows_df['cwe_id'].value_counts()
cwe_id_count

cwe_id
CWE-125    685
CWE-119    500
CWE-787    460
CWE-476    388
CWE-190    278
          ... 
CWE-16       1
CWE-312      1
CWE-320      1
CWE-182      1
CWE-460      1
Name: count, Length: 129, dtype: int64

In [17]:
df = vul_rows_df
top_10_categories = df['repo_url'].value_counts().index
result = df[df['repo_url'].isin(top_10_categories)].groupby('repo_url')['cwe_id'].value_counts()
result_df = result.unstack(fill_value=0)
result_df['Total'] = result_df.sum(axis=1)
result_df

cwe_id,CWE-1077,CWE-116,CWE-1187,CWE-119,CWE-120,CWE-121,CWE-122,CWE-125,CWE-1284,CWE-129,...,CWE-909,CWE-913,CWE-918,CWE-924,CWE-94,CWE-943,CWE-96,NVD-CWE-Other,NVD-CWE-noinfo,Total
repo_url,,,,,,,,,,,,,,,,,,,,,
https://github.com/01org/tpm2.0-tools,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
https://github.com/10gen-archive/mongo-c-driver-legacy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,36
https://github.com/389ds/389-ds-base,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26
https://github.com/ARMmbed/mbedtls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
https://github.com/AXDOOMER/doom-vanille,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://github.com/zeromq/libzmq,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
https://github.com/zhaozg/lua-openssl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
https://github.com/zmartzone/mod_auth_openidc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14


In [19]:
top_5_values = result_df.sum(axis=0).nlargest(21).index
result_df1 = result_df[top_5_values].copy()
result_df1

cwe_id,Total,CWE-125,CWE-119,CWE-787,CWE-476,CWE-190,NVD-CWE-noinfo,CWE-20,CWE-416,CWE-120,...,CWE-362,CWE-200,CWE-415,CWE-264,CWE-617,CWE-399,CWE-189,CWE-22,CWE-400,CWE-667
repo_url,,,,,,,,,,,,,,,,,,,,,
https://github.com/01org/tpm2.0-tools,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://github.com/10gen-archive/mongo-c-driver-legacy,36,0,0,0,0,36,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://github.com/389ds/389-ds-base,26,0,0,0,0,0,0,0,0,0,...,0,22,0,0,0,0,0,0,0,0
https://github.com/ARMmbed/mbedtls,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://github.com/AXDOOMER/doom-vanille,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://github.com/zeromq/libzmq,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://github.com/zhaozg/lua-openssl,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://github.com/zmartzone/mod_auth_openidc,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
repo_url = vul_rows_df['repo_url'].value_counts().head(10).index.tolist()


['https://github.com/torvalds/linux',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/radareorg/radare2',
 'https://github.com/gpac/gpac',
 'https://github.com/Oryx-Embedded/CycloneTCP',
 'https://github.com/ImageMagick/ImageMagick',
 'https://github.com/FFmpeg/FFmpeg',
 'https://github.com/php/php-src',
 'https://github.com/the-tcpdump-group/tcpdump',
 'https://github.com/krb5/krb5']

In [32]:
# Group by 'repo_url' and 'commit_type', and then count the number of commits
commit_counts_by_type = df2.groupby(['cwe_id', 'commit_type']).size().reset_index(name='count')

# Pivot the data so that 'commit_type' becomes columns
commit_counts_pivot = commit_counts_by_type.pivot_table(index='cwe_id', columns='commit_type', values='count', fill_value=0).reset_index()

# Display the pivoted results
commit_counts_pivot['commit_in_total'] = commit_counts_pivot.get('FC', 0) + commit_counts_pivot.get('VCC', 0)
commit_counts_pivot

commit_type,cwe_id,FC,VCC,commit_in_total
0,CWE-1021,1.0,5.0,6.0
1,CWE-1077,1.0,1.0,2.0
2,CWE-114,2.0,1.0,3.0
3,CWE-116,3.0,26.0,29.0
4,CWE-1187,1.0,0.0,1.0
...,...,...,...,...
135,CWE-94,6.0,16.0,22.0
136,CWE-943,1.0,2.0,3.0
137,CWE-96,1.0,2.0,3.0
138,NVD-CWE-Other,118.0,192.0,310.0


In [3]:
df1 = pd.read_csv('final_dataset/cve_fc_vcc_mapping.csv')
print(df1.shape)
df2 = pd.read_csv('final_dataset/commit_info.csv')
df1 = df1.drop_duplicates(subset='fc_hash')
print(df1.shape)
# Check if df2['hash'] is in df1['fc_hash']
matches = df2['hash'].isin(df1['fc_hash'])

matches

(4161, 5)
(3917, 5)


0         True
1        False
2        False
3        False
4        False
         ...  
11788    False
11789    False
11790    False
11791    False
11792    False
Name: hash, Length: 11793, dtype: bool